# FRD Accomplishments Script <br>
## This goal of this script is to get the total accomplishments for each category for each county using the ELMR report and UCF Report. The ELMR report is generated through microsoft access, and the UCF report is from Mac. <br>
### Read the README File in Github for more detailed information.  https://github.com/jgorman-tfs/FRD-Accomplishments

## Before you do prodeed, ensure you have done the following:
1. Import the new FIA report into the Microsoft Access Database
2. Generate the report
3. Clean the UCF Report, creating two new sheets for the ELMR and SPAM sheets. They should have 3 columns: City or County Name, Assist Type, and Category.
## Set you folder paths and qtr name here. You should not need to change anything else.

In [21]:
#import required modules
import os
import pandas as pd
from openpyxl import load_workbook

#folder and gdb path
folder_path = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2026Q1"
gdb = r"D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb"

#Quater and Year you are working in
qtr = "FY2026_Q1"

#Name of your county layer in the geodatabase - This will be a general Texas Counties layer with a NAME field
counties_template = "Template"

#Name of the Texas_Places_WithCounties layer
cities = "Texas_Places_WithCounties"

#Path to excel file generated from Microsoft Access
accomp_from_access = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2026Q1\Accomplishments.XLSX\qryAll$"

#excel path to the UCF diviion reporting containing the cleaned cities and new sheets
excel_path = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2026Q1\Q1FY26_Urban_Working.xlsx"

#Sheet names of edited data from Mac
sheet_name_ELMR = "UCF_Accomplishments_ELMR"
sheet_name_SPAM = "UCF_Accomplishments_SPAM"

#These are temporary files, no need to change them
ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')

In [22]:
#Create a copy of the County Layer that will be used for the map
counties_new = f"{qtr}_Counties"
arcpy.FeatureClassToFeatureClass_conversion(f"{gdb}\\{counties_template}", gdb, counties_new)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\FY2026_Q1_Counties'>

In [23]:
# Read data from the specified Excel sheets into dataframes
accomp_from_ELMR_df = pd.read_excel(excel_path, sheet_name=sheet_name_ELMR)
accomp_from_SPAM_df = pd.read_excel(excel_path, sheet_name=sheet_name_SPAM)

# Rename columns for ELMR data and clean the CountyName field
old_column_list_ELMR = accomp_from_ELMR_df.columns.tolist()
old_column_list_SPAM = accomp_from_SPAM_df.columns.tolist()

new_column_names_ELMR = {
    old_column_list_ELMR[0]: "CountyName",
    old_column_list_ELMR[1]: "AssistType",
    old_column_list_ELMR[2]: "Category"
}
accomp_from_ELMR_df.rename(columns=new_column_names_ELMR, inplace=True)
accomp_from_ELMR_df['CountyName'] = accomp_from_ELMR_df['CountyName'].str.replace(' ', '', regex=False)
print("ELMR: \n")
print(f"{accomp_from_ELMR_df.head(10)}\n")

# Rename columns for SPAM data and clean the CityName field
new_column_names_SPAM = {
    old_column_list_SPAM[0]: "CityName",
    old_column_list_SPAM[1]: "AssistType",
    old_column_list_SPAM[2]: "Category"
}
accomp_from_SPAM_df.rename(columns=new_column_names_SPAM, inplace=True)
accomp_from_SPAM_df['CityName'] = accomp_from_SPAM_df['CityName'].str.replace(' ', '', regex=False)
print("SPAM: \n")
print(accomp_from_SPAM_df.head(10))

ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')

# Convert the DataFrame to an Excel file
accomp_from_ELMR_df.to_csv(ELMR_accomp_temp, index=False)
accomp_from_SPAM_df.to_csv(SPAM_accomp_temp, index=False)

arcpy.TableToTable_conversion(ELMR_accomp_temp, gdb, f"{qtr}_Counties_FromELMR")
arcpy.TableToTable_conversion(SPAM_accomp_temp, gdb, f"{qtr}_Cities_FromSPAM")
arcpy.TableToTable_conversion(accomp_from_access, gdb, f"{qtr}_Counties_FromAccess")

# Join city data with their respective counties
in_data = f"{gdb}\\{qtr}_Cities_FromSPAM"
in_field = "CityName"
join_table = f"{gdb}\\{cities}"
join_field = "CityName"
fields = "CountyName"
arcpy.management.JoinField(in_data, in_field, join_table, join_field, fields)

# Merge city and county data
inputs = [f"{gdb}\\{qtr}_Cities_FromSPAM", f"{gdb}\\{qtr}_Counties_FromELMR" ]
output = f"{gdb}\\{qtr}_Counties_FromUCF_Merge"
arcpy.management.Merge(inputs, output)

# Generate a summary of counties and their assist types
output_table = f"{gdb}\\{qtr}_Counties_FromUCF"
arcpy.analysis.Statistics(output, output_table, [["CountyName", "COUNT"]], ["Category", "CountyName"])

# Ensure missing values in specific fields are set to zero
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Conservation_Education IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Management_Plans IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Management_Plans", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Technical_Assistance IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "FIA_Plots IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "FIA_Plots", 0 )

#clear selection
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "CLEAR_SELECTION")

# Add and calculate new fields to track matched counties
arcpy.management.AddField(f"{qtr}_Counties_FromAccess", "CountyChecker", "LONG")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "CountyChecker", 0 )
arcpy.management.JoinField(f"{qtr}_Counties_FromUCF", "CountyName", f"{qtr}_Counties_FromAccess" , "CountyName", "CountyChecker")

ELMR: 

   CountyName         AssistType                Category
0        Hays  Arbor Day Program  Conservation Education
1       Smith  Arbor Day Program  Conservation Education
2        Camp  Arbor Day Program  Conservation Education
3       Bexar  Arbor Day Program  Conservation Education
4   Somervell  Arbor Day Program  Conservation Education
5  Montgomery  Arbor Day Program  Conservation Education
6   PaloPinto  Arbor Day Program  Conservation Education
7      Parker  Arbor Day Program  Conservation Education
8        Kerr  Arbor Day Program  Conservation Education
9   Matagorda  Arbor Day Program  Conservation Education

SPAM: 

      CityName                                AssistType  \
0        Alamo  Education/Outreach Event or Presentation   
1        Alamo         Tree Planting/Maintenance Program   
2        Alice  Education/Outreach Event or Presentation   
3        Alton         Tree Planting/Maintenance Program   
4       Austin             Arbor Day/Tree City USA Event

<Result 'FY2026_Q1_Counties_FromUCF'>

## Critical Step
### Run the cell below to see if there are null values in the County Name field. If there are, it is likely because the CityName was actually reported as a County instead of a city. In that case, you can just delete those rows. It may also be null because of a misspelling. In that case, fix the spelling of the city. Re-run the cell below until there are no null values in the County Name column. 

### There will be null values in the CityName column because the ELMR table contains Counties only - so dont worry about those. Only focus on the County Name Column

In [25]:
# Check for unmatched counties and handle appropriately
null_count = 0
with arcpy.da.SearchCursor(f"{qtr}_Counties_FromUCF_Merge", ["CountyName"]) as cursor:
    for row in cursor:
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the Counties_FromUCF_Merge table. Go back and fix until there are no NULL values in the County Name column.")

All counties match, proceed to next step


In [27]:
arcpy.management.AddField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", 'LONG')

<Result 'FY2026_Q1_Counties_FromUCF'>

In [28]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Conservation Education'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromAccess", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Conservation_Education_UCF")

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Technical Assistance'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromAccess", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Technical_Assistance_UCF")
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "CLEAR_SELECTION")

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Technical_Assistance_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "NEW_SELECTION", "Conservation_Education_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromAccess", "CLEAR_SELECTION")

arcpy.management.AddField(f"{qtr}_Counties_FromAccess", "Conservation_Education_Total", 'LONG')
arcpy.management.AddField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_Total", 'LONG')
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_Total", 0)
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_Total", 0)

arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Conservation_Education_Total", "int(!Conservation_Education!) + int(!Conservation_Education_UCF!)")
arcpy.management.CalculateField(f"{qtr}_Counties_FromAccess", "Technical_Assistance_Total", "!Technical_Assistance! + !Technical_Assistance_UCF!")

arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromAccess" , 
                           "CountyName", 
                           "CountyChecker")

<Result 'FY2026_Q1_Counties'>

In [29]:
#Final check to for null values in the new county layer
null_count = 0
with arcpy.da.SearchCursor(counties_new, ["CountyChecker"]) as cursor:
    for row in cursor:
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the UCF table. Go back and fix before proceeding.")

All counties match, proceed to next step


In [30]:
#Join the final table to the county layer
arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromAccess" , 
                           "CountyName", 
                           ["FIA_Plots", "Management_Plans", "Conservation_Education_Total", "Technical_Assistance_Total"])

<Result 'FY2026_Q1_Counties'>